In [1]:
import requests
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import gamma
import numpy as np
from functools import reduce
import c3aidatalake

# At least for this EDA, let's just look at Georgia

ignore this bc it was way easier to get/format data straight from us census
ga_dem = c3aidatalake.fetch(
    'populationdata',
    {"spec": {
       "filter": "contains(parent, 'Georgia') && year == 2010 && estimate == 'False'"
        
    }},
    get_all = True
)
ga_dem.head()

In [10]:
ga_dem = pd.read_csv('../../data/demographics/georgia_county_dem.csv')
ga_dem['parent.id'] = ga_dem.apply(lambda x: x['CTYNAME'].split(' County')[0] + '_' + x['STNAME'] + '_UnitedStates', axis=1)
ga_dem = ga_dem[ga_dem['YEAR']==1]
year_desc = {0: 'Total'
,1: '0-4'
,2 : '5-9'
,3 : '10-14'
,4 : '15-19'
,5 : '20-24'
,6 : '25-29'
,7 : '30-34'
,8 : '35-39'
,9 : '40-44'
,10 : '45-49'
,11 : '50-54'
,12 : '55-59'
,13 : '60-64'
,14 : '65-69'
,15 : '70-74'
,16 : '75-79'
,17 : '80-84'
,18 : '85_plus'}
ga_dem['AGEGRP'] = ga_dem['AGEGRP'].replace(year_desc)
ga_dem.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE,parent.id
0,50,13,1,Georgia,Appling County,1,Total,18236,9159,9077,...,621,48,42,21,26,6,4,24,0,Appling_Georgia_UnitedStates
1,50,13,1,Georgia,Appling County,1,0-4,1306,647,659,...,106,7,7,3,3,1,0,1,0,Appling_Georgia_UnitedStates
2,50,13,1,Georgia,Appling County,1,5-9,1286,658,628,...,78,6,5,2,4,1,0,1,0,Appling_Georgia_UnitedStates
3,50,13,1,Georgia,Appling County,1,10-14,1331,703,628,...,63,1,7,3,2,1,1,4,0,Appling_Georgia_UnitedStates
4,50,13,1,Georgia,Appling County,1,15-19,1254,691,563,...,58,8,3,0,4,0,0,3,0,Appling_Georgia_UnitedStates


In [3]:
def get_JHU(column):
    today = pd.Timestamp.now().strftime("%Y-%m-%d")
    rst = c3aidatalake.evalmetrics(
        "outbreaklocation",
        {
            "spec" : {
                "filter": "contains(id, 'Georgia')",
                "expressions" : [column],
                "start" : today,
                "end" : today,
                "interval" : "DAY",
            }
        }
    )
    rst = rst.filter(regex=("\.data")).melt()
    rst["parent.id"] = rst['variable'].apply(lambda x: x.split('.')[0])
    del rst['variable']
    rst = rst.rename(columns={'value': column})
    rst.head()
    return rst

In [4]:
dem_data = [ga_dem]

In [5]:
cases = get_JHU('JHU_ConfirmedCases')
dem_data.append(cases)
cases.head()

,JHU_ConfirmedCases,parent.id
0,440,Johnson_Georgia_UnitedStates
1,210,Heard_Georgia_UnitedStates
2,2012,Rockdale_Georgia_UnitedStates
3,1826,Habersham_Georgia_UnitedStates
4,1187,Liberty_Georgia_UnitedStates


In [6]:
deaths = get_JHU('JHU_ConfirmedDeaths')
dem_data.append(deaths)
deaths.head()

,JHU_ConfirmedDeaths,parent.id
0,22,Johnson_Georgia_UnitedStates
1,6,Heard_Georgia_UnitedStates
2,40,Rockdale_Georgia_UnitedStates
3,74,Habersham_Georgia_UnitedStates
4,24,Liberty_Georgia_UnitedStates


In [7]:
recs = get_JHU('JHU_ConfirmedRecoveries')
dem_data.append(recs)
recs.head()

,JHU_ConfirmedRecoveries,parent.id
0,0,Johnson_Georgia_UnitedStates
1,0,Heard_Georgia_UnitedStates
2,0,Rockdale_Georgia_UnitedStates
3,0,Habersham_Georgia_UnitedStates
4,0,Liberty_Georgia_UnitedStates


In [8]:
ga = reduce(lambda left,right: pd.merge(left, right, on=['parent.id']), dem_data)
#ga = ga.drop(['year','gender','race','ethnicity','estimate','median'
#              ,'percent','minAge','maxAge','origin'
#             ,'timestamp', 'populationAge'],axis=1)
#ga['id'] = ga['id'].apply(lambda x: x.split('Male/Female_')[1])
#ga = ga[ga['id'].str.contains('Census')]
ga.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE,parent.id,JHU_ConfirmedCases,JHU_ConfirmedDeaths,JHU_ConfirmedRecoveries
0,50,13,1,Georgia,Appling County,1,0,18236,9159,9077,...,21,26,6,4,24,0,Appling_Georgia_UnitedStates,1129,34,0
1,50,13,1,Georgia,Appling County,1,1,1306,647,659,...,3,3,1,0,1,0,Appling_Georgia_UnitedStates,1129,34,0
2,50,13,1,Georgia,Appling County,1,2,1286,658,628,...,2,4,1,0,1,0,Appling_Georgia_UnitedStates,1129,34,0
3,50,13,1,Georgia,Appling County,1,3,1331,703,628,...,3,2,1,1,4,0,Appling_Georgia_UnitedStates,1129,34,0
4,50,13,1,Georgia,Appling County,1,4,1254,691,563,...,0,4,0,0,3,0,Appling_Georgia_UnitedStates,1129,34,0
